# Test PlaqueDataset vs Notebook Dataset

This notebook tests whether the training performance difference is due to:
1. Dataset differences (PlaqueDataset vs notebook's custom dataset)
2. Model architecture differences

We'll use the same model architecture as the notebook but with your PlaqueDataset.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from tqdm import tqdm
from PIL import Image
import albumentations as A
from typing import List, Dict

import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models

from concurrent.futures import ThreadPoolExecutor
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
import seaborn as sns

from typing import Optional, Any, Tuple
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    accuracy_score
)

# Import your PlaqueDataset
import sys
sys.path.append('src')
from models.data.plaque_dataset import PlaqueDataset, load_dataloaders
from models.config import Config
from utils import load_config


In [2]:
# Set parameters to match notebook
NUM_EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 1e-5
NUM_WORKERS = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cpu


In [3]:
# Load your PlaqueDataset using the same config as supervised_model.py
config = load_config("configs", "supervised")
train_loader, val_loader, test_loader, unlabeled_loader = load_dataloaders(config)

print(f"Train loader batches: {len(train_loader)}")
print(f"Val loader batches: {len(val_loader)}")
print(f"Test loader batches: {len(test_loader)}")

# Get a sample batch to check data format
sample_batch = next(iter(train_loader))
print(f"\nSample batch structure:")
print(f"Number of items in batch: {len(sample_batch)}")
for i, item in enumerate(sample_batch):
    if hasattr(item, 'shape'):
        print(f"Item {i}: shape {item.shape}, dtype {item.dtype}")
    else:
        print(f"Item {i}: {type(item)} - {item[:3] if len(item) > 3 else item}")


Preloading unlabeled plaque images...: 100%|██████████| 2000/2000 [00:20<00:00, 95.74it/s] 
Train loader batches: 12
Val loader batches: 3
Test loader batches: 4


c:\Users\DELL\Desktop\Honours\pbt-plaque-analysis\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Sample batch structure:
Number of items in batch: 7
Item 0: <class 'tuple'> - ('data\\labeled_images\\Diffuse\\Image_2018-086_lobparinf_AB4_index_3978.png', 'data\\labeled_images\\Coarse\\Image_2014-049_Occipital_AB_index_2336.png', 'data\\labeled_images\\Coarse\\Image_2021-075_occipitalA_AB4_index_7004.png')
Item 1: shape torch.Size([32, 3, 224, 224]), dtype torch.float32
Item 2: shape torch.Size([32, 3, 224, 224]), dtype torch.float32
Item 3: shape torch.Size([32, 3, 224, 224]), dtype torch.float32
Item 4: shape torch.Size([32, 3, 224, 224]), dtype torch.float32
Item 5: shape torch.Size([32, 0]), dtype torch.float32
Item 6: shape torch.Size([32]), dtype torch.int64


In [4]:
# Define the same SimpleCNN model as in the notebook
class SimpleCNN(nn.Module):
    def __init__(self, num_classes: int):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )

    def forward(self, x: torch.Tensor):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Create model
num_classes = len(config.name_to_label)
simple_model = SimpleCNN(num_classes=num_classes).to(device)
print(f"Model created with {num_classes} classes")
print(f"Total parameters: {sum(p.numel() for p in simple_model.parameters()):,}")


Model created with 9 classes
Total parameters: 25,785,929


In [5]:
# Test model with PlaqueDataset data
sample_batch = next(iter(train_loader))
(image_paths, scaled_raw_images, scaled_transformed_images, 
 scaled_normalized_raw_images, scaled_normalized_transformed_images, 
 extra_features, labels) = sample_batch

print(f"Input shapes:")
print(f"  scaled_normalized_transformed_images: {scaled_normalized_transformed_images.shape}")
print(f"  labels: {labels.shape}")
print(f"  Input range: [{scaled_normalized_transformed_images.min():.2f}, {scaled_normalized_transformed_images.max():.2f}]")

# Test forward pass
simple_model.eval()
with torch.no_grad():
    test_input = scaled_normalized_transformed_images[:4].to(device)
    test_output = simple_model(test_input)
    print(f"\nModel test:")
    print(f"  Input shape: {test_input.shape}")
    print(f"  Output shape: {test_output.shape}")
    print(f"  Output range: [{test_output.min():.2f}, {test_output.max():.2f}]")
    
    # Check predictions
    predictions = torch.argmax(test_output, dim=1)
    print(f"  Predictions: {predictions.cpu().numpy()}")
    print(f"  Unique predictions: {torch.unique(predictions).cpu().numpy()}")


Input shapes:
  scaled_normalized_transformed_images: torch.Size([32, 3, 224, 224])
  labels: torch.Size([32])
  Input range: [0.00, 255.00]

Model test:
  Input shape: torch.Size([4, 3, 224, 224])
  Output shape: torch.Size([4, 9])
  Output range: [-3.64, 7.22]
  Predictions: [3 3 3 3]
  Unique predictions: [3]


In [6]:
# Training function (same as notebook)
def train(
    model: nn.Module,
    train_loader: DataLoader,
    criterion: nn.Module,
    optimizer: optim.Optimizer,
    val_loader: Optional[DataLoader] = None,
    device: str = 'cpu',
    num_epochs: int = 10,
    early_stopping: bool = True,
    patience: int = 5
):
    model.train()
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        # Training phase
        model.train()
        for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")):
            # Extract data from PlaqueDataset batch format
            (image_paths, scaled_raw_images, scaled_transformed_images, 
             scaled_normalized_raw_images, scaled_normalized_transformed_images, 
             extra_features, labels) = batch
            
            # Use the same input as your supervised_model.py
            images = scaled_normalized_transformed_images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Print progress for first few batches
            if batch_idx < 3:
                print(f"Batch {batch_idx}: Loss={loss.item():.4f}, Acc={100.*correct/total:.2f}%")
                print(f"  Predictions: {predicted[:5].cpu().numpy()}")
                print(f"  Labels: {labels[:5].cpu().numpy()}")
        
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = 100. * correct / total
        
        # Validation phase
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        if val_loader is not None:
            model.eval()
            with torch.no_grad():
                for batch in val_loader:
                    (image_paths, scaled_raw_images, scaled_transformed_images, 
                     scaled_normalized_raw_images, scaled_normalized_transformed_images, 
                     extra_features, labels) = batch
                    
                    images = scaled_normalized_transformed_images.to(device)
                    labels = labels.to(device)
                    
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    
                    val_loss += loss.item() * images.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()
            
            val_loss = val_loss / len(val_loader.dataset)
            val_acc = 100. * val_correct / val_total
        else:
            val_loss = 0.0
            val_acc = 0.0
        
        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f} - Train Acc: {epoch_acc:.2f}% | Val Loss: {val_loss:.4f} - Val Acc: {val_acc:.2f}%")
        
        # Early stopping
        if early_stopping and val_loader is not None:
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break


In [7]:
# Train the model with PlaqueDataset
print("="*60)
print("TRAINING WITH PLAQUEDATASET")
print("="*60)

# Setup training
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(simple_model.parameters(), lr=LEARNING_RATE)

# Train for 5 epochs to compare with notebook
train(
    simple_model, 
    train_loader, 
    criterion, 
    optimizer, 
    val_loader=val_loader, 
    device=device, 
    num_epochs=5,  # Same as notebook comparison
    early_stopping=False
)


TRAINING WITH PLAQUEDATASET


Epoch 1/5:   8%|▊         | 1/12 [00:03<00:43,  3.98s/it]

Batch 0: Loss=5.0498, Acc=12.50%
  Predictions: [3 3 7 5 3]
  Labels: [0 3 3 3 5]


Epoch 1/5:  17%|█▋        | 2/12 [00:07<00:39,  3.95s/it]

Batch 1: Loss=7.7237, Acc=7.81%
  Predictions: [0 2 1 4 2]
  Labels: [1 7 2 0 1]


...

Batch 1: Loss=2.0994, Acc=17.19%
  Predictions: [0 1 1 1 1]
  Labels: [1 7 2 0 1]


Epoch 5/5:  25%|██▌       | 3/12 [00:08<00:25,  2.79s/it]

Batch 2: Loss=2.1683, Acc=18.75%
  Predictions: [2 0 5 1 5]
  Labels: [2 0 2 8 8]


Epoch 5/5: 100%|██████████| 12/12 [00:33<00:00,  2.78s/it]


Epoch 5/5 - Train Loss: 2.1370 - Train Acc: 15.96% | Val Loss: 2.1178 - Val Acc: 16.84%


In [ ]:
# Compare with notebook results
print("="*60)
print("COMPARISON WITH NOTEBOOK RESULTS")
print("="*60)
print("Notebook results (5 epochs):")
print("  Epoch 1: Train Acc: 15.43%")
print("  Epoch 2: Train Acc: 24.73%")
print("  Epoch 3: Train Acc: 35.11%")
print("  Epoch 4: Train Acc: 40.16%")
print("  Epoch 5: Train Acc: 45.74%")
print("\nYour PlaqueDataset results (5 epochs):")
print("  [Results will be shown above after training]")
print("\nAnalysis:")
print("  - If PlaqueDataset shows similar progress → Model architecture issue")
print("  - If PlaqueDataset shows different progress → Dataset issue")
